# Week 9 Lab: Building Interactive AI Demos

**CS 203: Software Tools and Techniques for AI**

In this lab, you'll learn to:
1. Build a Streamlit app from scratch
2. Add interactive widgets
3. Create a Gradio demo
4. Deploy to Hugging Face Spaces

**Note:** Parts of this lab involve creating separate Python files. You'll run those files in your terminal.

## Setup

In [ ]:
# Install required packages
# !pip install streamlit gradio pandas scikit-learn matplotlib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

print("Imports successful!")

## Part 1: Create and Save a Model

First, let's train a simple model that we'll use in our demo.

In [ ]:
# Create sample movie data
np.random.seed(42)
n_samples = 500

genres = np.random.choice(['Action', 'Comedy', 'Drama', 'Horror', 'Sci-Fi'], n_samples)
budgets = np.random.uniform(5, 300, n_samples)
runtimes = np.random.uniform(80, 180, n_samples)
is_sequel = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])

# Create target with logic
success = (
    (budgets > 100) & 
    (np.isin(genres, ['Action', 'Sci-Fi'])) |
    (is_sequel == 1) & (budgets > 50)
).astype(int)
# Add some noise
noise_idx = np.random.choice(n_samples, size=int(0.1 * n_samples), replace=False)
success[noise_idx] = 1 - success[noise_idx]

movies = pd.DataFrame({
    'genre': genres,
    'budget': budgets,
    'runtime': runtimes,
    'is_sequel': is_sequel,
    'success': success
})

print(f"Created dataset with {len(movies)} movies")
print(f"Success rate: {movies['success'].mean():.1%}")
movies.head()

In [ ]:
# Encode categorical and train model
le = LabelEncoder()
movies['genre_encoded'] = le.fit_transform(movies['genre'])

X = movies[['genre_encoded', 'budget', 'runtime', 'is_sequel']]
y = movies['success']

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

print(f"Model trained! Accuracy: {model.score(X, y):.1%}")

In [ ]:
# Save the model and encoder
with open('movie_model.pkl', 'wb') as f:
    pickle.dump({'model': model, 'encoder': le}, f)

print("Model saved to movie_model.pkl")

## Part 2: Your First Streamlit App

Create a file called `app_v1.py` with the content below.

Then run it with: `streamlit run app_v1.py`

In [ ]:
app_v1 = '''
import streamlit as st

# Title
st.title("Hello, Streamlit!")

# Text
st.write("This is my first Streamlit app.")

# Input
name = st.text_input("What's your name?")

# Output
if name:
    st.write(f"Hello, {name}!")
'''

# Save to file
with open('app_v1.py', 'w') as f:
    f.write(app_v1)

print("Created app_v1.py")
print("\nRun with: streamlit run app_v1.py")

## Part 3: Adding More Widgets

Let's create a more interactive version.

In [ ]:
app_v2 = '''
import streamlit as st

st.title("Streamlit Widgets Demo")

# Different input types
st.header("Input Widgets")

# Text
name = st.text_input("Your name")

# Number
age = st.number_input("Your age", min_value=0, max_value=120, value=25)

# Slider
rating = st.slider("Rate your experience", 1, 10, 5)

# Dropdown
color = st.selectbox("Favorite color", ["Red", "Blue", "Green", "Yellow"])

# Checkbox
subscribe = st.checkbox("Subscribe to newsletter")

# Button
if st.button("Submit"):
    st.success("Form submitted!")
    st.write(f"Name: {name}")
    st.write(f"Age: {age}")
    st.write(f"Rating: {rating}/10")
    st.write(f"Color: {color}")
    st.write(f"Subscribed: {subscribe}")
'''

with open('app_v2.py', 'w') as f:
    f.write(app_v2)

print("Created app_v2.py")
print("\nRun with: streamlit run app_v2.py")

## Part 4: Netflix Movie Predictor Demo

Now let's build the full Netflix predictor demo!

In [ ]:
app_netflix = '''
import streamlit as st
import pandas as pd
import pickle

# Page config
st.set_page_config(
    page_title="Netflix Movie Predictor",
    page_icon="🎬",
    layout="centered"
)

# Load model (with caching)
@st.cache_resource
def load_model():
    with open("movie_model.pkl", "rb") as f:
        return pickle.load(f)

# Load the model
data = load_model()
model = data["model"]
encoder = data["encoder"]

# Title and description
st.title("🎬 Netflix Movie Success Predictor")
st.markdown("""
Will your movie be a hit? Enter the details below to find out!

This model was trained on historical movie data to predict success.
""")

st.divider()

# Input section
st.header("📝 Movie Details")

col1, col2 = st.columns(2)

with col1:
    genre = st.selectbox(
        "Genre",
        ["Action", "Comedy", "Drama", "Horror", "Sci-Fi"],
        help="Select the primary genre"
    )
    
    budget = st.slider(
        "Budget (millions $)",
        min_value=1,
        max_value=300,
        value=50,
        help="Production budget in millions of dollars"
    )

with col2:
    runtime = st.slider(
        "Runtime (minutes)",
        min_value=60,
        max_value=240,
        value=120,
        help="Movie duration in minutes"
    )
    
    is_sequel = st.checkbox("Is this a sequel?", help="Check if this is part of a franchise")

st.divider()

# Prediction section
if st.button("🎯 Predict Success", type="primary"):
    # Prepare features
    genre_encoded = encoder.transform([genre])[0]
    features = pd.DataFrame({
        "genre_encoded": [genre_encoded],
        "budget": [budget],
        "runtime": [runtime],
        "is_sequel": [int(is_sequel)]
    })
    
    # Make prediction
    with st.spinner("Analyzing your movie..."):
        import time
        time.sleep(0.5)  # Simulate processing
        
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0]
    
    # Display results
    st.header("📊 Prediction Result")
    
    if prediction == 1:
        st.success("🎉 This movie is predicted to be a SUCCESS!")
        confidence = probability[1] * 100
    else:
        st.error("⚠️ This movie might struggle at the box office.")
        confidence = probability[0] * 100
    
    st.metric("Confidence", f"{confidence:.1f}%")
    
    # Show feature summary
    with st.expander("View movie details"):
        st.write(f"- Genre: {genre}")
        st.write(f"- Budget: ${budget}M")
        st.write(f"- Runtime: {runtime} minutes")
        st.write(f"- Sequel: {'Yes' if is_sequel else 'No'}")

# Footer
st.divider()
st.caption("Built with Streamlit | CS 203 - Week 9")
'''

with open('app_netflix.py', 'w') as f:
    f.write(app_netflix)

print("Created app_netflix.py")
print("\nRun with: streamlit run app_netflix.py")

## Part 5: Gradio Demo

Let's create the same predictor using Gradio.

In [ ]:
gradio_app = '''
import gradio as gr
import pandas as pd
import pickle

# Load model
with open("movie_model.pkl", "rb") as f:
    data = pickle.load(f)
    model = data["model"]
    encoder = data["encoder"]

def predict_success(genre, budget, runtime, is_sequel):
    """Predict movie success."""
    # Prepare features
    genre_encoded = encoder.transform([genre])[0]
    features = pd.DataFrame({
        "genre_encoded": [genre_encoded],
        "budget": [budget],
        "runtime": [runtime],
        "is_sequel": [int(is_sequel)]
    })
    
    # Predict
    prediction = model.predict(features)[0]
    probability = model.predict_proba(features)[0]
    
    # Format result
    if prediction == 1:
        result = f"🎉 SUCCESS! (Confidence: {probability[1]*100:.1f}%)"
    else:
        result = f"⚠️ Risky (Confidence: {probability[0]*100:.1f}%)"
    
    return result

# Create interface
demo = gr.Interface(
    fn=predict_success,
    inputs=[
        gr.Dropdown(["Action", "Comedy", "Drama", "Horror", "Sci-Fi"], label="Genre"),
        gr.Slider(1, 300, value=50, label="Budget (millions $)"),
        gr.Slider(60, 240, value=120, label="Runtime (minutes)"),
        gr.Checkbox(label="Is Sequel?")
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="🎬 Netflix Movie Success Predictor",
    description="Will your movie be a hit? Enter the details to find out!",
    examples=[
        ["Action", 200, 150, False],
        ["Comedy", 30, 100, False],
        ["Sci-Fi", 150, 140, True],
    ]
)

demo.launch()
'''

with open('app_gradio.py', 'w') as f:
    f.write(gradio_app)

print("Created app_gradio.py")
print("\nRun with: python app_gradio.py")

## Part 6: Try Both Apps!

Open two terminal windows and run:

**Terminal 1 (Streamlit):**
```bash
streamlit run app_netflix.py
```

**Terminal 2 (Gradio):**
```bash
python app_gradio.py
```

Compare the two approaches!

## Part 7: Deployment to Hugging Face Spaces

### Files needed for deployment:

1. `app.py` - Your application code
2. `requirements.txt` - Dependencies
3. `movie_model.pkl` - Your trained model

### Steps:

1. Go to https://huggingface.co and sign up
2. Click "New Space"
3. Choose "Streamlit" or "Gradio"
4. Clone the repository and add your files
5. Push to deploy!

In [ ]:
# Create requirements.txt for deployment
requirements = """
streamlit==1.28.0
pandas==2.0.0
scikit-learn==1.3.0
numpy==1.24.0
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements.strip())

print("Created requirements.txt for deployment")
print(requirements)

In [ ]:
# Create README for Hugging Face Space
readme = """
---
title: Netflix Movie Predictor
emoji: 🎬
colorFrom: red
colorTo: blue
sdk: streamlit
python_version: 3.10
---

# Netflix Movie Success Predictor

Predicts whether a movie will be successful based on:
- Genre
- Budget
- Runtime
- Whether it's a sequel

Built with Streamlit for CS 203.
"""

with open('README.md', 'w') as f:
    f.write(readme.strip())

print("Created README.md for Hugging Face Space")

## Part 8: Summary and Exercises

In [ ]:
print("""
Summary of what we created:
============================

Files created:
- movie_model.pkl    : Trained model
- app_v1.py          : Basic Streamlit app
- app_v2.py          : Streamlit with more widgets
- app_netflix.py     : Full Netflix predictor (Streamlit)
- app_gradio.py      : Netflix predictor (Gradio)
- requirements.txt   : Dependencies for deployment
- README.md          : Hugging Face Space config

To run:
- Streamlit: streamlit run app_netflix.py
- Gradio:    python app_gradio.py

Exercises:
1. Add more features to the predictor (director, actors, etc.)
2. Add a data visualization section
3. Deploy your app to Hugging Face Spaces
4. Add feedback buttons (thumbs up/down)
5. Create a multi-page Streamlit app
""")

In [ ]:
# Clean up files (optional - run this if you want to remove created files)
# import os
# for f in ['app_v1.py', 'app_v2.py', 'app_netflix.py', 'app_gradio.py', 
#           'movie_model.pkl', 'requirements.txt', 'README.md']:
#     if os.path.exists(f):
#         os.remove(f)
#         print(f"Removed {f}")